In [10]:
import pandas as pd
import requests
import json
import time
import datetime
from tqdm import tqdm

In [13]:
def getComments(course_id, page):
    """
    return json檔
    """
    payload = {'courseId': course_id,
           'page': page, 
           'is_text_review': 1,
           'ordering': 'course_review_score__rank,-created', 
           'fields[course_review]': '@default,response,created_formatted_with_time_since', 
           'fields[course_review_response]': '@min,user,created_formatted_with_time_since'}
    url = 'https://www.udemy.com/api-2.0/courses/684272/reviews/'
    r = requests.get(url, params=payload)
    if r.status_code == 200:
        api_data = r.json()
        return api_data

In [14]:
"""
input: Course_id
output: 20 comments
"""
def UGC(course_id):
    output = []
    i = 0
    page = 0
    while (i<100):
        api_data = getComments(course_id, page+1)
        #先比對時間是否在3/17之前 有的話再繼續
        latest_date_timestamp = time.mktime(datetime.datetime.strptime('2022-03-17', "%Y-%m-%d").timetuple())
        
        for data in api_data['results']:
            #紀錄1.commentid, 2.contnet, 3.user, 4.created, 5. ifResponse(1:true, 0:false) 6.response
            temp = []
            comment_date_timestamp = time.mktime(datetime.datetime.strptime(data['created'][:10], "%Y-%m-%d").timetuple())
            #若在3/17之後 直接下一筆
            if comment_date_timestamp > latest_date_timestamp:
                continue
            
            #print('%d/%d' % (i+1, 100))
            
            #0.course_id
            temp.append(course_id)
            #1.commentId
            temp.append(data['id'])
            #1.5 rating
            temp.append(data['rating'])
            #2.content
            temp.append(data['content'])
            #3.user
            temp.append(data['user']['title'])
            #4.creted
            temp.append(data['created'])
            
            if data['response']==None:
                temp.append(0)
                temp.append(None)
            else:
                temp.append(1)
                temp.append(data['response'])
            
            output.append(temp)
            i += 1
            
            #檢查是否有100則
            if i==100:
                return output
            
        page += 1

In [11]:
output = []
for course in tqdm(courses):
    result = UGC(course)
    output.append(result)

df = pd.DataFrame(output)
df.to_excel('UGC-332.xlsx')

 76%|██████████████████████████████████████████████████████████████▎                   | 57/75 [06:53<02:10,  7.25s/it]


KeyboardInterrupt: 